In [54]:
import pandas as pd
import numpy as np
from datetime import datetime


agency = pd.read_csv('Data/INTERCITE/agency.txt')
feed_info = pd.read_csv('Data/INTERCITE/feed_info.txt')
transfers = pd.read_csv('Data/INTERCITE/transfers.txt')
calendar_dates = pd.read_csv('Data/INTERCITE/calendar_dates.txt')
routes = pd.read_csv('Data/INTERCITE/routes.txt')
stop_times = pd.read_csv('Data/INTERCITE/stop_times.txt')
stops = pd.read_csv('Data/INTERCITE/stops.txt')
trips = pd.read_csv('Data/INTERCITE/trips.txt')

In [71]:
stops.fillna('', inplace=True)
calendar_dates.date = pd.to_datetime(calendar_dates["date"], format='%Y%m%d').dt.date
calendar_dates


,service_id,date,exception_type
0,1,2024-06-30,1
1,2,2024-06-29,1
2,2,2024-07-06,1
3,2,2024-07-07,1
4,2,2024-07-13,1
...,...,...,...
5781,0,2024-09-20,1
5782,0,2024-09-21,1
5783,0,2024-09-22,1
5784,0,2024-09-23,1


In [166]:
# Exemple de ville : StopArea:OCE87586545

class AnalyzerGTFS:
    stop_ids = [] #Identifiants des StopsPoints de la ville
    trip_ids = [] #Identifiants des Trajets liés à tous les StopPoints de la ville
    service_id = [] #Identifiants des services faisant des trajets à des dates comprises entre date_min et date_max
    stop_time_trip = [] # Contient StopPoints, Trajets et stop_sequence
    trip = [] # Contient les trip_id, route_id et les services_id
    tripDates = []  # Contient les trajets des services fonctionnant sur les dates données
    stops_trip_id = [] # Contient tous les stops des trajets corrects
    date = [] # Contient tous les services et toutes les dates comprises entre date_min et date_max, (Utile pour afficher à l'utilisateur toutes les dates possibles pour le trajet)
    destinations = [] # Contient les stops après la ville (StopSequence > StopSequence de la ville)
    def __init__(self, id_ville, date_min,date_max):
        self.id_ville = id_ville
        self.date_min = datetime.strptime(date_min, '%Y%m%d').date()
        self.date_max = datetime.strptime(date_max, '%Y%m%d').date()
    
    # Etape 1 : Récupérer les stops de la ville / StopArea
    def get_stops(self):
        self.stop_ids = stops[stops['parent_station'].str.contains(self.id_ville)]['stop_id']
        return self.stop_ids
    
    # Etape 2 : Récupérer les trajets des StopPoints, peut importe la date
    def get_trips(self):
        self.stop_time_trip = pd.merge(stop_times, self.stop_ids, on='stop_id')
        self.trip_ids = self.stop_time_trip['trip_id']
        return self.trip_ids   

    # Etape 3 : Récupérer les services_id des Trajets
    def get_service_id(self):
        self.trip = pd.merge(self.trip_ids, trips, on='trip_id')
        self.service_id = self.trip['service_id']
        return self.service_id 
    
    # Etape 4 : Récupérer les services fonctionnant sur les dates données
    def get_dates(self):
        service_id = self.get_service_id()
        dates = pd.merge(service_id, calendar_dates, on='service_id')
        self.date = dates[(dates['date'] >= self.date_min) & (dates['date'] <= self.date_max)]
        return self.date
    
    # Etape 5 : Récupérer les trajets des services fonctionnant sur les dates données
    def get_trajets(self):
        services_uniques = self.date.drop_duplicates(subset='service_id')
        self.trajets = pd.merge(services_uniques, self.trip, on='service_id')
        return self.trajets
    
    # Etape 6 : Récupérer lles stops des trajets corrects
    def get_stops_trajets(self):
        self.stops_trip_id = pd.merge(self.trajets, stop_times, on='trip_id')
        return self.stops_trip_id
    
    # Etape 7 : Récupérer les destinations, c'est à dire les stops après la ville (StopSequence > StopSequence de la ville)
    def get_stops_destinations(self):
        self.destinations = self.stops_trip_id[self.stops_trip_id['stop_sequence'] > self.stops_trip_id[self.stops_trip_id['stop_id'] == self.id_ville]['stop_sequence']]
        return self.destinations

In [177]:
Analyzer = AnalyzerGTFS('StopArea:OCE87586545','20240601','20240731')
Analyzer.get_stops()
Analyzer.get_trips()
Analyzer.get_service_id()
Analyzer.get_dates()
Analyzer.get_trajets()
Analyzer.get_stops_trajets()

Analyzer.date

,service_id,date,exception_type
0,227,2024-07-01,1
1,227,2024-07-02,1
2,227,2024-07-03,1
3,227,2024-07-04,1
4,227,2024-07-05,1
...,...,...,...
289,275,2024-07-31,1
340,110,2024-07-07,1
341,110,2024-07-14,1
342,110,2024-07-21,1


In [165]:
# Analyzer.stop_time_trip.index = Analyzer.stop_time_trip['trip_id']
# Analyzer.stop_time_trip.sort_index(axis=1,inplace=True)
# Analyzer.stops_trip_id.index = Analyzer.stops_trip_id['trip_id']
# Analyzer.stops_trip_id.sort_index(axis=1,inplace=True)
Analyzer.stop_time_trip
#Analyzer.stops_trip_id
citytrip_idANDsequence = pd.merge(Analyzer.stops_trip_id, Analyzer.stop_time_trip, on=['stop_id','trip_id'])
citytrip_idANDsequence
#Analyzer.stops_trip_id[Analyzer.stops_trip_id['stop_sequence'] > citytrip_idANDsequence['stop_sequence'] & Analyzer.stops_trip_id['trip_id'] == citytrip_idANDsequence['trip_id']]
#df.loc[df['shield'] > 6, ['max_speed']]

,service_id,date,exception_type,trip_id,route_id,trip_headsign,direction_id,block_id,shape_id,arrival_time_x,...,pickup_type_x,drop_off_type_x,shape_dist_traveled_x,arrival_time_y,departure_time_y,stop_sequence_y,stop_headsign_y,pickup_type_y,drop_off_type_y,shape_dist_traveled_y
0,227,2024-07-01,1,OCESN4657F3063601:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,592,NaN,09:04:00,...,0,0,NaN,09:04:00,09:06:00,1,NaN,0,0,NaN
1,227,2024-07-01,1,OCESN4764F3088783:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4764,0.0,684,NaN,20:57:00,...,0,0,NaN,20:57:00,20:59:00,8,NaN,0,0,NaN
2,5,2024-06-26,1,OCESN4657F3067278:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,593,NaN,08:51:00,...,0,0,NaN,08:51:00,08:53:00,1,NaN,0,0,NaN
3,199,2024-06-29,1,OCESN4657F3067539:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,594,NaN,08:49:00,...,0,0,NaN,08:49:00,08:51:00,1,NaN,0,0,NaN
4,231,2024-07-06,1,OCESN4657F3088822:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,595,NaN,09:04:00,...,0,0,NaN,09:04:00,09:06:00,1,NaN,0,0,NaN
5,232,2024-07-08,1,OCESN4657F3088823:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,596,NaN,09:04:00,...,0,0,NaN,09:04:00,09:06:00,1,NaN,0,0,NaN
6,37,2024-07-01,1,OCESN4669F3088799:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4669,1.0,632,NaN,19:04:00,...,0,0,NaN,19:04:00,19:06:00,1,NaN,0,0,NaN
7,37,2024-07-01,1,OCESN4758F3152970:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4758,0.0,658,NaN,14:57:00,...,0,0,NaN,14:57:00,14:59:00,8,NaN,0,0,NaN
8,184,2024-07-05,1,OCESN4669F3088800:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4669,1.0,633,NaN,19:04:00,...,0,0,NaN,19:04:00,19:06:00,1,NaN,0,0,NaN
9,184,2024-07-05,1,OCESN4758F3152971:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4758,0.0,659,NaN,14:57:00,...,0,0,NaN,14:57:00,14:59:00,8,NaN,0,0,NaN
